In [1]:
import pandas as pd
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

ModuleNotFoundError: No module named 'pandas'

In [2]:
import pandas as pd
from pyspark.sql import SparkSession

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
import yaml
import sqlalchemy
import pandas as pd

from sqlalchemy import text

with open('Sensitive data/db_credentials.yaml', 'r') as file:
    credentials = yaml.safe_load(file)

aws_engine = sqlalchemy.create_engine(
            f"mysql+pymysql://{credentials['USER']}"
            f":{credentials['PASSWORD']}"
            f"@{credentials['HOST']}"
            f":{credentials['PORT']}"
            f"/{credentials['DATABASE']}?charset=utf8mb4"
        )

def return_table(sql_querry):
    with aws_engine.connect() as connection:
        querry = text(sql_querry)
        table = connection.execute(querry)
        df = pd.DataFrame(table)
        return spark.createDataFrame(df)

In [ ]:
pin_df = return_table("SELECT * FROM pinterest_data")

In [ ]:
geo_df = return_table("SELECT * FROM geolocation_data")

In [ ]:
user_df = return_table("SELECT * FROM user_data")

In [ ]:
pin_df_null_descriptions = pin_df.replace(['No description available Story format', 'User Info Error', 'Image src error.', 'N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e', 'No Title Data Available', 'Untitled', 'multi-video(story page format)'], None)
pin_df_null_values = pin_df_null_descriptions.replace(0, None, subset=['downloaded'])
from pyspark.sql.functions import regexp_replace
pin_df_follower_count_k = pin_df_null_values.withColumn("follower_count", regexp_replace("follower_count", "k", "000"))
pin_df_follower_count_m = pin_df_follower_count_k.withColumn("follower_count", regexp_replace("follower_count", "M", "000000"))
pin_df_fc_datatype_int = pin_df_follower_count_m.withColumn("follower_count", pin_df_follower_count_m["follower_count"].cast("integer"))
pin_df_ind_datatype_int = pin_df_fc_datatype_int.withColumn("index", pin_df_follower_count_m["index"].cast("integer"))
pin_df_localsave_clean = pin_df_ind_datatype_int.withColumn("save_location", regexp_replace("save_location", "Local save in ", ""))
df_pin_rename_column = pin_df_localsave_clean.withColumnRenamed("index", "ind")
df_pin_clean = df_pin_rename_column.select("ind", "unique_id", "title", "description", "follower_count", "poster_name", "tag_list", "is_image_or_video", "image_src", "save_location", "category")


In [ ]:
df_pin_clean.show()

+---+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+-------------+--------+
|ind|           unique_id|               title|         description|follower_count|         poster_name|            tag_list|is_image_or_video|           image_src|save_location|category|
+---+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+-------------+--------+
|  0|31af679a-6e4f-426...|Vanessa-Stockard-...|Few would argue i...|       2000000|         Bored Panda|Pretty Art,Cute A...|            image|https://i.pinimg....|    /data/art|     art|
|  1|79dbcedb-4f96-4fc...|Meet The Enigmati...|This Turkish, Ist...|       2000000|         Bored Panda|Surealism Art,Sur...|            image|https://i.pinimg....|    /data/art|     art|
|  2|f0c7a5c5-9a0a-478...|Art Colorful Feat...|Challenging s

In [ ]:
pd_df = geo_df_clean.toPandas()

In [ ]:
from pyspark.sql.functions import array
from pyspark.sql.functions import to_timestamp

geo_df_new_column = geo_df.withColumn("coordinates", concat("latitude", "longitude"))
geo_df_drop_column = geo_df_new_column.drop("latitude", "longitude")
geo_df_to_timestamp = geo_df_drop_column.withColumn("timestamp", to_timestamp("timestamp"))
geo_df_clean = geo_df_to_timestamp.select("ind", "country", "coordinates", "timestamp")

In [ ]:
from pyspark.sql.functions import concat, lit
from pyspark.sql.functions import to_timestamp

user_df_concat_names = user_df.withColumn("user_name", concat("first_name", lit(" "), "last_name"))
user_df_drop_col = user_df_concat_names.drop("first_name", "last_name")
user_df_to_timestamp = user_df_drop_col.withColumn("date_joined", to_timestamp("date_joined"))
user_df_clean = user_df_to_timestamp.select("ind", "user_name", "age", "date_joined")

In [ ]:
localengine = sqlalchemy.create_engine("sqlite:///pint_db.db")
pd_df.to_sql('geo_df', localengine, if_exists='replace')

11154